In [1]:
import time
import argparse
import numpy as np

import torch
import torch.optim as optim

import torch.nn as nn

from utils import load_data,normalize,toy_data,nmi_score,modularity_matrix,modularity
from models import GNN

import community as community_louvain
from networkx import from_numpy_matrix
import networkx as nx

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

from sklearn.cluster import SpectralClustering

import pycombo

import pandas as pd

torch.set_printoptions(sci_mode=False)

from utils import doublerelu

import networkx as nx

In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
cuda = torch.cuda.is_available()
weight_decay = 10e-2
epochs = 1000
seed = 17

In [4]:
class GNN1Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN1Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size

        weight1_eye = torch.FloatTensor(torch.eye(in_features, out_features))
        weight1_eye = weight1_eye.reshape((1, in_features, out_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        self.weight1 = Parameter(weight1_eye)
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [5]:
class GNN2Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN2Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size
        weight1_eye = torch.FloatTensor(torch.eye(in_features, in_features))
        weight1_eye = weight1_eye.reshape((1, in_features, in_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        weight1_rand = torch.empty(batch_size,in_features,out_features-in_features)
        torch.nn.init.xavier_uniform_(weight1_rand, gain=1.0)
        self.weight1 = Parameter(torch.cat((weight1_eye,weight1_rand),dim=2))
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [6]:
class GNN1(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN1, self).__init__()

        self.gc1 = GNN1Layer(batch_size, ndim, ndim)

    def forward(self, x, adj):
        x = doublerelu(self.gc1(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        return x

In [7]:
class GNN2(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN2, self).__init__()

        self.gc1 = GNN2Layer(batch_size, ndim, hidden)
        self.gc2 = GNN1Layer(batch_size, hidden, ndim)

    def forward(self, x, adj):
        x = doublerelu(self.gc1(x, adj))
        x = doublerelu(self.gc2(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        return x

In [8]:
def community_detection(G,method,GNN,features=None,hidden=1,seed=seed):
    
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed(seed)
    
    adj = nx.to_numpy_matrix(G)
    adj = np.expand_dims(adj, axis=0)
        
    #Combo
    combo_partition = pycombo.execute(G)
    nb_comm = max(combo_partition[0].values())+1
    combo_mod = combo_partition[1]

    adj_norm = normalize(adj)

    adj = torch.FloatTensor(np.array(adj))
    adj_norm = torch.FloatTensor(np.array(adj_norm))
    
        
    if type(features) == type(None):

        # features
        if method == "louvain":
            partition = community_louvain.best_partition(G)
            #get binary matrix of the partition
            nb_community = max(list(partition.values())) + 1
            communities =  np.array(list(partition.values())).reshape(-1)
            C = np.eye(nb_community)[communities]
            C = torch.FloatTensor(C)
            #perturbations
            C = perturb(C)
        elif method == "spectral":
            partition = spectralPartition(adj,nb_comm)
            C = partition.float()
            nb_community = C.shape[1]

        features = torch.FloatTensor(C)
        features = features.unsqueeze(0)
    
    if method == "louvain":
        lr = 0.002
    elif method == "spectral":
        lr = 0.002
        
    nb_community = features.shape[-1]
    
    Q = modularity_matrix(adj)
    
    # Model and optimizer

    model = GNN(batch_size=adj.shape[0],
                nfeat=adj.shape[1],
                ndim=nb_community,
                hidden=hidden)

    if cuda:
        model.cuda()
        features = features.cuda()
        adj = adj.cuda()
        adj_norm = adj_norm.cuda()
        Q = Q.cuda()

    # Train model
    t_total = time.time()

    optimizer = optim.Adam(model.parameters(),
                           lr=lr, weight_decay=weight_decay)
    

    for epoch in range(epochs):

        t = time.time()
        model.train()
        optimizer.zero_grad()

        C = model(features, adj_norm)
        loss = modularity(C,Q)

        loss.backward(retain_graph=True)

        optimizer.step()

        if epoch == 0:
            best_loss = loss
            init_mod = loss
            best_C = C
        else:
            if loss < best_loss:
                best_loss = loss
                best_C = C

        if epoch % 100 == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'Modularity: {:.8f}'.format(-best_loss.item()),
                  'time: {:.4f}s'.format(time.time() - t))

    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    
    return nb_community,-init_mod,-best_loss,combo_mod, best_C

In [9]:
def perturb(A):
    p = 0.4
    epsilon = torch.rand(A.shape).uniform_(0, p) - torch.rand(A.shape).uniform_(0, p)
    return torch.clip(A + epsilon,0,1)

In [10]:
def spectralPartition(A,nb_comm):
    A = A[0] # TODO 3D
    clustering = SpectralClustering(n_clusters=nb_comm,assign_labels='discretize',random_state=0).fit(A)
    clusters = list(clustering.labels_)
    partitions = torch.tensor(clusters)
    return torch.nn.functional.one_hot(partitions)

In [11]:
def graphs():
    G = [nx.read_pajek('data/davidcopperfield.net'),nx.read_pajek('data/jazz.net'),nx.read_pajek('data/dolphins.net')]
    return G

In [12]:
def exp1(hidden):
    G = graphs()
    methods = ["louvain","spectral"]
    finetuned_mods = []
    networks = ["David Copperfield","David Copperfield","Jazz Musicians","Jazz Musicians","Dolphins Social Network","Dolphins Social Network"]
    for network in G:
        for method in methods:
            nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN1,hidden=hidden)
            finetuned_mods.append(finetuned_mod.cpu().detach().numpy()[0][0])
            
    return finetuned_mods

In [13]:
def exp2(hidden):
    G = graphs()
    methods = ["louvain","spectral"]
    nb_communities = []
    init_mods = []
    finetuned_mods = []
    combo_mods = []
    methodss = []
    networks = ["David Copperfield","David Copperfield","Jazz Musicians","Jazz Musicians","Dolphins Social Network","Dolphins Social Network"]
    hidden_dim = []
    one_layer_mod = exp1(hidden)
    for network in G:
        for method in methods:
            flag = True
            for i in range(1):
                seed = np.random.randint(10000)
                nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN1,hidden=hidden,seed=seed)
                initial_mod = torch.clone(init_mod)
                nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN2,features=features,hidden=nb_community+hidden,seed=seed)
                if flag:
                    nb_communities.append(nb_community)
                    init_mods.append(initial_mod.cpu().detach().numpy()[0][0])
                    finetuned_mods.append(finetuned_mod.cpu().detach().numpy()[0][0])
                    combo_mods.append(combo_mod)
                    methodss.append(method)
                    hidden_dim.append(nb_community+hidden)
                    flag = False
                else:
                    if finetuned_mod.cpu().detach().numpy()[0][0] > finetuned_mods[-1]:
                        finetuned_mods[-1] = finetuned_mod.cpu().detach().numpy()[0][0]
            
            
    df = pd.DataFrame([networks,methodss,nb_communities,hidden_dim,init_mods,one_layer_mod,finetuned_mods,combo_mods]).transpose()
    df.columns = ["Network","Method","Communities","Hidden Dimensionality","Initial Modularity","1 Layer Finetuned Moduclarity","2 Layer Finetuned Moduclarity","Combo Modularity"] 
    return df

In [14]:
dfs = exp2(1)
for hidden in range(2,11):
    dfs = pd.concat([dfs,exp2(hidden)])

Epoch: 0001 Modularity: 0.11401027 time: 0.3465s
Epoch: 0101 Modularity: 0.27859122 time: 0.0020s
Epoch: 0201 Modularity: 0.28211001 time: 0.0020s
Epoch: 0301 Modularity: 0.28426713 time: 0.0020s
Epoch: 0401 Modularity: 0.28530049 time: 0.0020s
Epoch: 0501 Modularity: 0.28600383 time: 0.0020s
Epoch: 0601 Modularity: 0.28602374 time: 0.0020s
Epoch: 0701 Modularity: 0.28602397 time: 0.0020s
Epoch: 0801 Modularity: 0.28602469 time: 0.0020s
Epoch: 0901 Modularity: 0.28602469 time: 0.0030s
Optimization Finished!
Total time elapsed: 2.2667s
Epoch: 0001 Modularity: 0.03314324 time: 0.0020s
Epoch: 0101 Modularity: 0.06331336 time: 0.0020s
Epoch: 0201 Modularity: 0.08956972 time: 0.0020s
Epoch: 0301 Modularity: 0.11138584 time: 0.0020s
Epoch: 0401 Modularity: 0.11138584 time: 0.0020s
Epoch: 0501 Modularity: 0.11138584 time: 0.0020s
Epoch: 0601 Modularity: 0.11138584 time: 0.0020s
Epoch: 0701 Modularity: 0.11138584 time: 0.0020s
Epoch: 0801 Modularity: 0.11138584 time: 0.0020s
Epoch: 0901 Modula

Epoch: 0201 Modularity: 0.50935483 time: 0.0040s
Epoch: 0301 Modularity: 0.50935483 time: 0.0030s
Epoch: 0401 Modularity: 0.50935483 time: 0.0040s
Epoch: 0501 Modularity: 0.50935483 time: 0.0040s
Epoch: 0601 Modularity: 0.50935483 time: 0.0030s
Epoch: 0701 Modularity: 0.50935483 time: 0.0040s
Epoch: 0801 Modularity: 0.50935483 time: 0.0030s
Epoch: 0901 Modularity: 0.50935483 time: 0.0030s
Optimization Finished!
Total time elapsed: 3.2258s
Epoch: 0001 Modularity: 0.23339660 time: 0.0020s
Epoch: 0101 Modularity: 0.26042399 time: 0.0010s
Epoch: 0201 Modularity: 0.27343124 time: 0.0020s
Epoch: 0301 Modularity: 0.31383947 time: 0.0020s
Epoch: 0401 Modularity: 0.35315388 time: 0.0020s
Epoch: 0501 Modularity: 0.35315388 time: 0.0020s
Epoch: 0601 Modularity: 0.35315388 time: 0.0020s
Epoch: 0701 Modularity: 0.35315388 time: 0.0020s
Epoch: 0801 Modularity: 0.35315388 time: 0.0020s
Epoch: 0901 Modularity: 0.35315388 time: 0.0010s
Optimization Finished!
Total time elapsed: 1.7922s
Epoch: 0001 Modu

Epoch: 0401 Modularity: 0.11545191 time: 0.0010s
Epoch: 0501 Modularity: 0.11545191 time: 0.0020s
Epoch: 0601 Modularity: 0.11545191 time: 0.0020s
Epoch: 0701 Modularity: 0.11545191 time: 0.0020s
Epoch: 0801 Modularity: 0.11545191 time: 0.0020s
Epoch: 0901 Modularity: 0.11545191 time: 0.0020s
Optimization Finished!
Total time elapsed: 1.5619s
Epoch: 0001 Modularity: 0.11545191 time: 0.0040s
Epoch: 0101 Modularity: 0.23238929 time: 0.0040s
Epoch: 0201 Modularity: 0.23862252 time: 0.0030s
Epoch: 0301 Modularity: 0.23862252 time: 0.0030s
Epoch: 0401 Modularity: 0.23862252 time: 0.0040s
Epoch: 0501 Modularity: 0.23862252 time: 0.0030s
Epoch: 0601 Modularity: 0.23862252 time: 0.0040s
Epoch: 0701 Modularity: 0.23862252 time: 0.0030s
Epoch: 0801 Modularity: 0.23862252 time: 0.0030s
Epoch: 0901 Modularity: 0.23862252 time: 0.0030s
Optimization Finished!
Total time elapsed: 3.4126s
Epoch: 0001 Modularity: 0.26544726 time: 0.0010s
Epoch: 0101 Modularity: 0.50538939 time: 0.0020s
Epoch: 0201 Modu

Epoch: 0601 Modularity: 0.16616502 time: 0.0040s
Epoch: 0701 Modularity: 0.16616502 time: 0.0030s
Epoch: 0801 Modularity: 0.16616502 time: 0.0040s
Epoch: 0901 Modularity: 0.16616502 time: 0.0040s
Optimization Finished!
Total time elapsed: 3.2872s
Epoch: 0001 Modularity: 0.26196504 time: 0.0020s
Epoch: 0101 Modularity: 0.43163443 time: 0.0020s
Epoch: 0201 Modularity: 0.43240243 time: 0.0020s
Epoch: 0301 Modularity: 0.43296632 time: 0.0010s
Epoch: 0401 Modularity: 0.43391028 time: 0.0010s
Epoch: 0501 Modularity: 0.43409517 time: 0.0020s
Epoch: 0601 Modularity: 0.43424094 time: 0.0020s
Epoch: 0701 Modularity: 0.43425614 time: 0.0010s
Epoch: 0801 Modularity: 0.43425640 time: 0.0010s
Epoch: 0901 Modularity: 0.43425664 time: 0.0020s
Optimization Finished!
Total time elapsed: 1.6496s
Epoch: 0001 Modularity: 0.43425757 time: 0.0020s
Epoch: 0101 Modularity: 0.43425757 time: 0.0040s
Epoch: 0201 Modularity: 0.43425757 time: 0.0030s
Epoch: 0301 Modularity: 0.43425757 time: 0.0030s
Epoch: 0401 Modu

Epoch: 0801 Modularity: 0.29386172 time: 0.0020s
Epoch: 0901 Modularity: 0.29386172 time: 0.0010s
Optimization Finished!
Total time elapsed: 1.2547s
Epoch: 0001 Modularity: 0.29386172 time: 0.0030s
Epoch: 0101 Modularity: 0.29386663 time: 0.0020s
Epoch: 0201 Modularity: 0.29386663 time: 0.0020s
Epoch: 0301 Modularity: 0.29386735 time: 0.0020s
Epoch: 0401 Modularity: 0.29387254 time: 0.0030s
Epoch: 0501 Modularity: 0.29389375 time: 0.0030s
Epoch: 0601 Modularity: 0.29392907 time: 0.0020s
Epoch: 0701 Modularity: 0.29416192 time: 0.0030s
Epoch: 0801 Modularity: 0.29416227 time: 0.0030s
Epoch: 0901 Modularity: 0.29416227 time: 0.0020s
Optimization Finished!
Total time elapsed: 2.3641s
Epoch: 0001 Modularity: 0.03314324 time: 0.0010s
Epoch: 0101 Modularity: 0.06331336 time: 0.0010s
Epoch: 0201 Modularity: 0.08956972 time: 0.0010s
Epoch: 0301 Modularity: 0.11138584 time: 0.0010s
Epoch: 0401 Modularity: 0.11138584 time: 0.0020s
Epoch: 0501 Modularity: 0.11138584 time: 0.0020s
Epoch: 0601 Modu

...

Epoch: 0601 Modularity: 0.35315388 time: 0.0020s
Epoch: 0701 Modularity: 0.35315388 time: 0.0020s
Epoch: 0801 Modularity: 0.35315388 time: 0.0020s
Epoch: 0901 Modularity: 0.35315388 time: 0.0020s
Optimization Finished!
Total time elapsed: 1.3624s
Epoch: 0001 Modularity: 0.35315388 time: 0.0030s
Epoch: 0101 Modularity: 0.39113116 time: 0.0030s
Epoch: 0201 Modularity: 0.42283559 time: 0.0030s
Epoch: 0301 Modularity: 0.42283559 time: 0.0030s
Epoch: 0401 Modularity: 0.42283559 time: 0.0020s
Epoch: 0501 Modularity: 0.42283559 time: 0.0020s
Epoch: 0601 Modularity: 0.42283559 time: 0.0020s
Epoch: 0701 Modularity: 0.42283559 time: 0.0030s
Epoch: 0801 Modularity: 0.42283559 time: 0.0020s
Epoch: 0901 Modularity: 0.42283559 time: 0.0020s
Optimization Finished!
Total time elapsed: 2.3959s
Epoch: 0001 Modularity: 0.11401027 time: 0.0020s
Epoch: 0101 Modularity: 0.27859122 time: 0.0020s
Epoch: 0201 Modularity: 0.28211001 time: 0.0020s
Epoch: 0301 Modularity: 0.28426713 time: 0.0010s
Epoch: 0401 Modu

Epoch: 0801 Modularity: 0.23259011 time: 0.0030s
Epoch: 0901 Modularity: 0.23259011 time: 0.0020s
Optimization Finished!
Total time elapsed: 2.3599s
Epoch: 0001 Modularity: 0.26544726 time: 0.0010s
Epoch: 0101 Modularity: 0.50538939 time: 0.0010s
Epoch: 0201 Modularity: 0.50794470 time: 0.0010s
Epoch: 0301 Modularity: 0.50906670 time: 0.0010s
Epoch: 0401 Modularity: 0.50935483 time: 0.0010s
Epoch: 0501 Modularity: 0.50935483 time: 0.0010s
Epoch: 0601 Modularity: 0.50935483 time: 0.0010s
Epoch: 0701 Modularity: 0.50935483 time: 0.0010s
Epoch: 0801 Modularity: 0.50935483 time: 0.0010s
Epoch: 0901 Modularity: 0.50935483 time: 0.0020s
Optimization Finished!
Total time elapsed: 1.2282s
Epoch: 0001 Modularity: 0.50935483 time: 0.0020s
Epoch: 0101 Modularity: 0.50935483 time: 0.0020s
Epoch: 0201 Modularity: 0.50935483 time: 0.0030s
Epoch: 0301 Modularity: 0.50935483 time: 0.0040s
Epoch: 0401 Modularity: 0.50935483 time: 0.0040s
Epoch: 0501 Modularity: 0.50935495 time: 0.0030s
Epoch: 0601 Modu

Epoch: 0101 Modularity: 0.43425757 time: 0.0030s
Epoch: 0201 Modularity: 0.43425757 time: 0.0030s
Epoch: 0301 Modularity: 0.43425757 time: 0.0020s
Epoch: 0401 Modularity: 0.43425757 time: 0.0020s
Epoch: 0501 Modularity: 0.43425763 time: 0.0020s
Epoch: 0601 Modularity: 0.43425763 time: 0.0020s
Epoch: 0701 Modularity: 0.43425763 time: 0.0020s
Epoch: 0801 Modularity: 0.43425763 time: 0.0020s
Epoch: 0901 Modularity: 0.43425763 time: 0.0020s
Optimization Finished!
Total time elapsed: 2.3144s
Epoch: 0001 Modularity: 0.06414337 time: 0.0020s
Epoch: 0101 Modularity: 0.07505678 time: 0.0020s
Epoch: 0201 Modularity: 0.08530867 time: 0.0010s
Epoch: 0301 Modularity: 0.11545191 time: 0.0010s
Epoch: 0401 Modularity: 0.11545191 time: 0.0010s
Epoch: 0501 Modularity: 0.11545191 time: 0.0010s
Epoch: 0601 Modularity: 0.11545191 time: 0.0010s
Epoch: 0701 Modularity: 0.11545191 time: 0.0020s
Epoch: 0801 Modularity: 0.11545191 time: 0.0020s
Epoch: 0901 Modularity: 0.11545191 time: 0.0020s
Optimization Finis

Epoch: 0401 Modularity: 0.11138584 time: 0.0010s
Epoch: 0501 Modularity: 0.11138584 time: 0.0020s
Epoch: 0601 Modularity: 0.11138584 time: 0.0010s
Epoch: 0701 Modularity: 0.11138584 time: 0.0010s
Epoch: 0801 Modularity: 0.11138584 time: 0.0010s
Epoch: 0901 Modularity: 0.11138584 time: 0.0010s
Optimization Finished!
Total time elapsed: 1.2559s
Epoch: 0001 Modularity: 0.11138585 time: 0.0020s
Epoch: 0101 Modularity: 0.13056773 time: 0.0030s
Epoch: 0201 Modularity: 0.15785655 time: 0.0030s
Epoch: 0301 Modularity: 0.16680332 time: 0.0030s
Epoch: 0401 Modularity: 0.16680332 time: 0.0020s
Epoch: 0501 Modularity: 0.16680332 time: 0.0030s
Epoch: 0601 Modularity: 0.16680332 time: 0.0020s
Epoch: 0701 Modularity: 0.16680332 time: 0.0020s
Epoch: 0801 Modularity: 0.16680332 time: 0.0020s
Epoch: 0901 Modularity: 0.16680332 time: 0.0030s
Optimization Finished!
Total time elapsed: 2.3250s
Epoch: 0001 Modularity: 0.26196504 time: 0.0010s
Epoch: 0101 Modularity: 0.43163443 time: 0.0010s
Epoch: 0201 Modu

Epoch: 0701 Modularity: 0.35315388 time: 0.0010s
Epoch: 0801 Modularity: 0.35315388 time: 0.0010s
Epoch: 0901 Modularity: 0.35315388 time: 0.0010s
Optimization Finished!
Total time elapsed: 1.2927s
Epoch: 0001 Modularity: 0.11128143 time: 0.0020s
Epoch: 0101 Modularity: 0.28521264 time: 0.0030s
Epoch: 0201 Modularity: 0.28928506 time: 0.0020s
Epoch: 0301 Modularity: 0.29241121 time: 0.0010s
Epoch: 0401 Modularity: 0.29316127 time: 0.0010s
Epoch: 0501 Modularity: 0.29337925 time: 0.0020s
Epoch: 0601 Modularity: 0.29346836 time: 0.0010s
Epoch: 0701 Modularity: 0.29359460 time: 0.0010s
Epoch: 0801 Modularity: 0.29386172 time: 0.0010s
Epoch: 0901 Modularity: 0.29386172 time: 0.0010s
Optimization Finished!
Total time elapsed: 1.2996s
Epoch: 0001 Modularity: 0.29386172 time: 0.0030s
Epoch: 0101 Modularity: 0.29386628 time: 0.0020s
Epoch: 0201 Modularity: 0.29386628 time: 0.0020s
Epoch: 0301 Modularity: 0.29386783 time: 0.0020s
Epoch: 0401 Modularity: 0.29387152 time: 0.0030s
Epoch: 0501 Modu

In [15]:
dfs

,Network,Method,Communities,Hidden Dimensionality,Initial Modularity,1 Layer Finetuned Moduclarity,2 Layer Finetuned Moduclarity,Combo Modularity
0,David Copperfield,louvain,7,8,0.111281,0.286025,0.294162,0.307540
1,David Copperfield,spectral,6,7,0.033143,0.111386,0.170386,0.302674
2,Jazz Musicians,louvain,4,5,0.261965,0.428760,0.434260,0.444469
3,Jazz Musicians,spectral,3,4,0.064143,0.115452,0.237873,0.444469
4,Dolphins Social Network,louvain,5,6,0.265447,0.508564,0.509355,0.526799
5,Dolphins Social Network,spectral,4,5,0.233397,0.353154,0.466695,0.526799
0,David Copperfield,louvain,7,9,0.111281,0.286025,0.294162,0.306907
1,David Copperfield,spectral,6,8,0.033143,0.107196,0.178174,0.304424
2,Jazz Musicians,louvain,4,6,0.261965,0.428760,0.434260,0.444469
3,Jazz Musicians,spectral,3,5,0.064143,0.115452,0.238623,0.444469
